In [31]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [32]:
import sqlalchemy

Server = "localhost"
Database = "Homeprices_Baku"
Driver = "ODBC Driver 17 for SQL Server" # in order to get this info, click Win + S and write 'odbc', then open drivers menu and scroll down
engine = sqlalchemy.create_engine(f'mssql://@{Server}/{Database}?driver={Driver}')

In [33]:
tbl = pd.read_sql('Select * from Houses', engine)
tbl

,ID,Location,Rooms,Area,Price
0,1,Səbail r.,5.0,120.0,127000
1,2,Nəsimi r.,9.0,450.0,1950000
2,3,Nardaran q.,7.0,513.0,765000
3,4,Masazır q.,6.0,220.0,215000
4,5,Hövsan q.,4.0,120.0,88000
...,...,...,...,...,...
11727,11728,Biləcəri q.,3.0,80.0,68000
11728,11729,Badamdar q.,5.0,200.0,330000
11729,11730,Binə q.,3.0,80.0,31000
11730,11731,Qara Qarayev m.,8.0,700.0,980000


In [34]:
tbl.isna().sum()

ID            0
Location      0
Rooms       892
Area          0
Price         0
dtype: int64

In [35]:
tbl.dropna(inplace = True)

In [36]:
tbl.dtypes

ID            int64
Location     object
Rooms       float64
Area        float64
Price         int64
dtype: object

In [37]:
df = tbl.drop(['ID'], axis = 'columns')
df

,Location,Rooms,Area,Price
0,Səbail r.,5.0,120.0,127000
1,Nəsimi r.,9.0,450.0,1950000
2,Nardaran q.,7.0,513.0,765000
3,Masazır q.,6.0,220.0,215000
4,Hövsan q.,4.0,120.0,88000
...,...,...,...,...
11727,Biləcəri q.,3.0,80.0,68000
11728,Badamdar q.,5.0,200.0,330000
11729,Binə q.,3.0,80.0,31000
11730,Qara Qarayev m.,8.0,700.0,980000


In [38]:
location_stats = df.groupby('Location')['Location'].agg('count').sort_values(ascending = False)
location_stats

Location
Mərdəkan q.    1912
Masazır q.      672
Şüvəlan q.      573
Badamdar q.     538
Binə q.         500
               ... 
Zərdab            1
Zuğulba q.        1
Ağsu              1
Ağstafa           1
Massiv V q.       1
Name: Location, Length: 168, dtype: int64

In [39]:
location_stats_less_than_10 = location_stats[location_stats <= 10]
location_stats_less_than_10

Location
Dədə Qorqud q.        10
Ağ şəhər q.           10
İsmayıllı             10
7-ci mikrorayon q.    10
8-ci kilometr q.      10
                      ..
Zərdab                 1
Zuğulba q.             1
Ağsu                   1
Ağstafa                1
Massiv V q.            1
Name: Location, Length: 70, dtype: int64

In [40]:
df['price_per_sqft'] = df['Price'] / df['Area']
df

,Location,Rooms,Area,Price,price_per_sqft
0,Səbail r.,5.0,120.0,127000,1058.333333
1,Nəsimi r.,9.0,450.0,1950000,4333.333333
2,Nardaran q.,7.0,513.0,765000,1491.228070
3,Masazır q.,6.0,220.0,215000,977.272727
4,Hövsan q.,4.0,120.0,88000,733.333333
...,...,...,...,...,...
11727,Biləcəri q.,3.0,80.0,68000,850.000000
11728,Badamdar q.,5.0,200.0,330000,1650.000000
11729,Binə q.,3.0,80.0,31000,387.500000
11730,Qara Qarayev m.,8.0,700.0,980000,1400.000000


In [41]:
df['Location'] = df['Location'].apply(lambda x : 'other' if x in location_stats_less_than_10 else x)
df[df['Location'] == 'other']

,Location,Rooms,Area,Price,price_per_sqft
7,other,17.0,350.0,365000,1042.857143
66,other,6.0,600.0,850000,1416.666667
202,other,6.0,200.0,170000,850.000000
214,other,3.0,40.0,14500,362.500000
318,other,8.0,120.0,47000,391.666667
...,...,...,...,...,...
11581,other,5.0,114.0,43000,377.192982
11583,other,5.0,181.0,62500,345.303867
11606,other,4.0,200.0,250000,1250.000000
11607,other,4.0,110.0,66000,600.000000


In [42]:
df['price_per_sqft'].describe()

count     10840.000000
mean       1257.028091
std        1477.270879
min           7.000000
25%         688.524590
50%        1050.000000
75%        1562.500000
max      100000.000000
Name: price_per_sqft, dtype: float64

In [43]:
[*df.groupby('Location')]

[('20 Yanvar m.',
             Location  Rooms   Area    Price  price_per_sqft
  155    20 Yanvar m.    5.0  340.0   127000      373.529412
  507    20 Yanvar m.    5.0  340.0   127000      373.529412
  787    20 Yanvar m.    5.0  340.0   127000      373.529412
  1756   20 Yanvar m.    3.0   60.0    47000      783.333333
  2072   20 Yanvar m.    5.0  340.0   127000      373.529412
  2508   20 Yanvar m.   10.0  570.0   800000     1403.508772
  2794   20 Yanvar m.    5.0  330.0  1200000     3636.363636
  2856   20 Yanvar m.    5.0  340.0   127000      373.529412
  3569   20 Yanvar m.    9.0  650.0  2200000     3384.615385
  4110   20 Yanvar m.    3.0   60.0    47000      783.333333
  4386   20 Yanvar m.    3.0   60.0    47000      783.333333
  6523   20 Yanvar m.   12.0  800.0   495000      618.750000
  6767   20 Yanvar m.    7.0  153.0   110000      718.954248
  6790   20 Yanvar m.    3.0   60.0    65000     1083.333333
  7030   20 Yanvar m.    5.0  340.0   127000      373.529412
  7656

In [44]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('Location'):
        m = np.mean(subdf['price_per_sqft'])
        sd = np.std(subdf['price_per_sqft'])
        reduced_df = subdf[(subdf["price_per_sqft"] > (m - sd)) & (subdf["price_per_sqft"] <= (m + sd))]
        df_out = pd.concat([df_out, reduced_df], ignore_index = True)
    return df_out
df = remove_pps_outliers(df)
df.shape

(8960, 5)

In [45]:
def remove_room_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('Location'):
        room_stats = {}
        for room, room_df in location_df.groupby('Rooms'):
            room_stats[room] = {
                'mean' : np.mean(room_df['price_per_sqft']),
                'std' : np.std(room_df['price_per_sqft']),
                'count' : room_df.shape[0]
            }
        for room, room_df in location_df.groupby('Rooms'):
            stats = room_stats.get(room - 1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(exclude_indices, room_df[room_df['price_per_sqft'] < stats['mean']].index.values)
    return df.drop(exclude_indices, axis = 'index')

df = remove_room_outliers(df)
df.shape

(5777, 5)

In [46]:
df['Rooms'].unique()

array([ 5.,  3., 10., 12.,  7.,  2.,  6.,  9.,  4.,  8., 15., 14., 20.,
       11., 13.,  1., 18., 16., 17.])

In [47]:
df[df['Rooms'] > 10]

,Location,Rooms,Area,Price,price_per_sqft
9,20 Yanvar m.,12.0,800.0,495000,618.750000
106,Abşeron r.,15.0,1000.0,1300000,1300.000000
203,Avtovağzal m.,14.0,576.0,650000,1128.472222
220,Avtovağzal m.,14.0,576.0,720000,1250.000000
244,Azadlıq Prospekti m.,20.0,1000.0,1650000,1650.000000
...,...,...,...,...,...
8672,Şüvəlan q.,15.0,950.0,950000,1000.000000
8696,Şüvəlan q.,12.0,750.0,1200000,1600.000000
8845,Şıxov q.,11.0,530.0,636000,1200.000000
8894,Əhmədli m.,11.0,500.0,339000,678.000000


In [48]:
df.drop(['price_per_sqft'], axis = 'columns', inplace = True)
df

,Location,Rooms,Area,Price
0,20 Yanvar m.,5.0,340.0,127000
1,20 Yanvar m.,5.0,340.0,127000
2,20 Yanvar m.,5.0,340.0,127000
3,20 Yanvar m.,3.0,60.0,47000
4,20 Yanvar m.,5.0,340.0,127000
...,...,...,...,...
8955,Əmircan q.,3.0,80.0,75000
8956,Əmircan q.,6.0,190.0,165000
8957,Əmircan q.,8.0,400.0,420000
8958,Əmircan q.,3.0,70.0,65000


In [49]:
df1 = pd.get_dummies(data = df, columns = ['Location'])
df1.drop(['Location_other'], axis = 'columns', inplace = True)
df1

,Rooms,Area,Price,Location_20 Yanvar m.,Location_20-ci sahə q.,Location_Abşeron r.,Location_Albalılıq q.,Location_Avtovağzal m.,Location_Azadlıq Prospekti m.,Location_Badamdar q.,...,Location_İnşaatçılar m.,Location_İçəri Şəhər m.,Location_Şamaxı,Location_Şağan q.,Location_Şüvəlan q.,Location_Şıxov q.,Location_Şəki,Location_Əhmədli m.,Location_Əhmədli q.,Location_Əmircan q.
0,5.0,340.0,127000,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5.0,340.0,127000,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,5.0,340.0,127000,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3.0,60.0,47000,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5.0,340.0,127000,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8955,3.0,80.0,75000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
8956,6.0,190.0,165000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
8957,8.0,400.0,420000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
8958,3.0,70.0,65000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [50]:
df1.replace({True : 1, False : 0}, inplace = True)
df1

,Rooms,Area,Price,Location_20 Yanvar m.,Location_20-ci sahə q.,Location_Abşeron r.,Location_Albalılıq q.,Location_Avtovağzal m.,Location_Azadlıq Prospekti m.,Location_Badamdar q.,...,Location_İnşaatçılar m.,Location_İçəri Şəhər m.,Location_Şamaxı,Location_Şağan q.,Location_Şüvəlan q.,Location_Şıxov q.,Location_Şəki,Location_Əhmədli m.,Location_Əhmədli q.,Location_Əmircan q.
0,5.0,340.0,127000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5.0,340.0,127000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5.0,340.0,127000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,60.0,47000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5.0,340.0,127000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8955,3.0,80.0,75000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8956,6.0,190.0,165000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8957,8.0,400.0,420000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8958,3.0,70.0,65000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [51]:
x = df1.drop(['Price'], axis = 'columns')
y = df1['Price']

In [54]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 10)

In [57]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(x_train, y_train)
lr_clf.score(x_test, y_test)

0.7025469457091273